# Make sess class for each session and save as pickle

Currently uses info from sessions_dict.py to loop through sessions and create the sess class.

sess pickle files will be named `<scene>_<session>_<scan>.pickle`  \
and saved in `path_dict['preprocessed_root']/sess/<animal>/<date>`.

Set `overwrite` to `True` if you want to overwrite existing .pickle files. Otherwise, you will get an error that the file already exists.

In [1]:
overwrite = False

In [2]:
import os
import numpy as np

from reward_relative import preprocessing as pp
from reward_relative import utilities as ut

import TwoPUtils


%load_ext autoreload
%autoreload 2

### Specify your path dictionary here.

In [3]:
from reward_relative.path_dict_firebird import path_dictionary as path_dict
path_dict

{'preprocessed_root': '/data/2P',
 'sbx_root': '/mnt/oak/InVivoDA/2P_Data',
 'gdrive_root': '/mnt/gdrive/2P_Data',
 'VR_Data': '/data/2P/VR_Data',
 'git_repo_root': '/home/mari/local_repos/2p_repos',
 'TwoPUtils': '/home/mari/local_repos/2p_repos/TwoPUtils',
 'home': '/home/mari',
 'fig_dir': '/data/2P/fig_scratch'}

## Scroll or click to the desired section for:

[Single plane](#Single-plane-sessions)

[Multi plane](#Multi-plane-sessions)


Within each section, define animal and iterate through sessions.

While running the below cells, if you get an error that says `DatabaseError: Execution failed on sql 'SELECT * FROM data': no such table: data`,
check that all of your .sqlite files are named properly and have data in them (i.e. `Scene_1.sqlite` instead of `'Scene_1(1).sqlite'`


# Single plane sessions

In [4]:
from reward_relative.sessions_dict import single_plane

In [6]:
## Define animal
animal = 'GCAMP15'
days = np.arange(0, len(single_plane[animal])) # range of days
days =days[1:3]
days

array([1, 2])

In [7]:
basedir = os.path.join(path_dict['preprocessed_root'], animal)
sbxdir = os.path.join(path_dict['gdrive_root'], animal) #os.path.join(path_dict['sbx_root'], animal) # ['gdrive_root']
vrdir = path_dict['VR_Data']

binary_from_sbxdir = True # only relevant for downsampling
calcium_exists = True

load_suite2p = True
load_scaninfo = True
VR_only = False

trial_matrix_kwargs = []

for i, day in enumerate(days):

    if type(single_plane[animal][day]) is not tuple:
        date = single_plane[animal][day]['date']
        scene = single_plane[animal][day]['scene']
        session = single_plane[animal][day]['session']
        scan_number = single_plane[animal][day]['scan']

        sess = pp.create_sess(basedir, sbxdir, vrdir, animal, date, scene, session, scan_number,
                              load_scaninfo=load_scaninfo,
                              load_VR=True,
                              load_suite2p=load_suite2p,
                              load_behavior=True,
                              VR_only=VR_only,                              
                              )

        sess_dir = os.path.join(
            path_dict['preprocessed_root'], 'sess', animal, date)
        os.makedirs(sess_dir, exist_ok=True)
        print(sess_dir)

        if np.isnan(scan_number):
            scan_number=0
            
        sess_name = '%s_%03d_%03d.pickle' % (scene,
                                             session,
                                             scan_number
                                             )
        # Write sess to pickle file
        ut.write_sess_pickle(sess, sess_dir, sess_name, overwrite=overwrite)

    else:
        print("Iterating through multiple sessions")
        for i in range(len(single_plane[animal][day])):
            date = single_plane[animal][day][i]['date']
            scene = single_plane[animal][day][i]['scene']
            session = single_plane[animal][day][i]['session']
            scan_number = single_plane[animal][day][i]['scan']

            sess = pp.create_sess(basedir, sbxdir, vrdir, animal, date, scene, session, scan_number,
                                  load_scaninfo=True,
                                  load_VR=True,
                                  load_suite2p=True,
                                  load_behavior=True)

            sess_dir = os.path.join(
                path_dict['preprocessed_root'], 'sess', animal, date)
            os.makedirs(sess_dir, exist_ok=True)
            print(sess_dir)

            sess_name = '%s_%03d_%03d.pickle' % (scene,
                                                 session,
                                                 scan_number,
                                                 )
            # Write sess to pickle file
            ut.write_sess_pickle(
                sess, sess_dir, sess_name, overwrite=overwrite)

Found 0 planes from scan info
Overwriting with kwarg nplanes = 1
Fixing teleports
frame rate 15.5078125
(111462,) ttl times,(23961,) ca2+ frame times
last time: VR 1545.1188916876574, ca2+ 1545.0277078085642
(23961, 18)
/data/InVivoDA/GCAMP15/25_03_2024/Env1_LocationA/Env1_LocationA_002_001/suite2p
/data/InVivoDA/sess/GCAMP15/25_03_2024
writing Env1_LocationA_002_001.pickle


NotImplementedError: .pickle already exists, aborting save. Set overwrite=True to overwrite.

# Multi plane sessions

In [12]:
from reward_relative.sessions_dict import multi_plane
#multi_plane

In [15]:
animal = 'GCAMP18'
days = np.arange(0, len(multi_plane[animal])) # range of days
nplanes = 2
days = days[1:18] #[2:4]
days

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17])

In [ ]:
basedir = os.path.join(path_dict['preprocessed_root'],animal)
sbxdir = os.path.join(path_dict['gdrive_root'], animal) #os.path.join(path_dict['sbx_root'],animal) 
vrdir = path_dict['VR_Data']

# Get data binary from basedir or sbxdir?
binary_from_sbxdir = False
calcium_exists = True
add_suite2p = True

for day in days: 
# day=0
    if type(multi_plane[animal][day]) is not tuple:   
        date = multi_plane[animal][day]['date']
        scene = multi_plane[animal][day]['scene']
        session = multi_plane[animal][day]['session']
        scan_number = multi_plane[animal][day]['scan']

        fullpath = os.path.join(basedir,date,scene,"%s_%03d_%03d" % (scene, session, scan_number))
        scanpath = os.path.join(sbxdir,date,scene,"%s_%03d_%03d" % (scene, session, scan_number)) #change back to sbxdir

        sess = pp.create_sess(basedir,sbxdir,vrdir,animal,date,scene,session,scan_number,
                               load_scaninfo=True,
                               load_VR = True,
                               load_suite2p = add_suite2p,
                               load_behavior = True)

        nframes = int(sess.scan_info['max_idx']/sess.n_planes)

        sess_dir = os.path.join(path_dict['preprocessed_root'],'sess',animal,date)

        os.makedirs(sess_dir,exist_ok=True)
        print(sess_dir)

        sess_name = '%s_%03d_%03d.pickle' % (scene, 
                                             session,
                                             scan_number,
                                             )
        # Write sess to pickle file
        ut.write_sess_pickle(sess,sess_dir,sess_name,overwrite=overwrite)

    else:
        print("Iterating through multiple sessions")
        for i in range(len(multi_plane[animal][day])):
            date = multi_plane[animal][day][i]['date']
            scene = multi_plane[animal][day][i]['scene']
            session = multi_plane[animal][day][i]['session']
            scan_number = multi_plane[animal][day][i]['scan']

            fullpath = os.path.join(basedir,date,scene,"%s_%03d_%03d" % (scene, session, scan_number))
            scanpath = os.path.join(sbxdir,date,scene,"%s_%03d_%03d" % (scene, session, scan_number))

            sess = pp.create_sess(basedir,sbxdir,vrdir,animal,date,scene,session,scan_number,
                       load_scaninfo=True,
                       load_VR = True,
                       load_suite2p = add_suite2p,
                       load_behavior = True)

            nframes = int(sess.scan_info['max_idx']/sess.n_planes)

            sess_dir = os.path.join(path_dict['preprocessed_root'],'sess',animal,multi_plane[animal][day][i]['date'])

            os.makedirs(sess_dir,exist_ok=True)
            print(sess_dir)

            sess_name = '%s_%03d_%03d.pickle' % (scene, 
                                                 session,
                                                 scan_number,
                                                 )
            # Write sess to pickle file
            ut.write_sess_pickle(sess,sess_dir,sess_name,overwrite=overwrite)
